In [1]:
%load_ext autoreload
%autoreload 2

### test QE analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvcircuit as pvc
import ipywidgets as widgets
%matplotlib widget

Tandems version 0.989
Data path is:  /Users/jgeisz/opt/anaconda3/envs/pvdev/lib/python3.9/site-packages/tandems/data/


In [3]:
#import EQE of MM927 4J
path = pvc.datapath

file = 'MM927Bn5CEQE.csv'
df4J  = pd.read_csv(path+file, index_col=0)
MM927_EQE = df4J.to_numpy(dtype=np.float64, copy=True) #numpy.ndarray
MM927_xEQE = df4J.index.to_numpy(dtype=np.float64, copy=True) #numpy.ndarray

EQE4J = pvc.EQE(MM927_EQE, MM927_xEQE, 'MM927_EQE')
plt.ioff()

In [4]:
ui, ax, rax = EQE4J.controls(Pspec='direct')
ui